In [ ]:
import numpy as np
import rasterio
import fiona
from fiona.crs import from_epsg
import os
import shutil
import time
from vis_tools import read_hysplit_netcdf

model = read_hysplit_netcdf(FILENAME)

container = FILENAME.rstrip('.nc')
os.makedirs(container)  

    
##########################################################
# RASTER #################################################
##########################################################
rast_dir = os.path.join(container, 'raster')
os.makedirs(rast_dir)   
    
# read netCDF (remove the trivial, empty t=0 step and transpose for GeoTIFF coord order)
temp = model.isel(time=slice(1, None)).transpose('time', 'lat', 'lon')

nodata = -47
np.place(temp['total_deposition'].values, np.isnan(temp['total_deposition'].values), nodata)  # transform all NaN's to nodata

temp.to_netcdf('temp.nc')
src = rasterio.open('temp.nc', 'r')
os.remove('temp.nc')

for time_step in src.indexes:
    
    t_i = time.time()
    
    timestamp = str(np.datetime64(temp['time'].values[time_step-1], 'h'))
    
    fname = timestamp+'.tif'
    raster = rasterio.open(os.path.join(rast_dir, fname),
                           'w', driver='GTiff',
                           height=src.height, width=src.width,
                           count=1, dtype=src.profile['dtype'],
                           crs={'init': 'epsg:4326'}, transform=src.affine,
                           nodata=nodata)
    raster.write(src.read(time_step), 1)  
    shared_profile = raster.profile
    raster.close()
    print('{} written'.format(fname))
    
    t_f = time.time()
    print('{:.2f} seconds elapsed\n'.format(t_f - t_i))

num_files = src.count
src.close()

print('{} GeoTIFF files written to {}\n'.format(num_files, rast_dir+'/'))


##########################################################
# SHAPEFILE ##############################################
##########################################################
volc_name = model.attrs['volcano'].lower().replace(' ', '_')

shp_dir = os.path.join(container, volc_name+'_shp')
os.makedirs(shp_dir)  

with fiona.open(os.path.join(shp_dir, volc_name+'.shp'), 'w', crs=from_epsg(4326), driver='ESRI Shapefile',
                schema={'geometry': 'Point', 'properties': {k: 'str' for k in model.attrs}}) as output:    

    point = {'type': 'Point', 'coordinates': tuple(model.attrs['volcano_location'][::-1])}

    output.write({'geometry': point, 'properties': {k: str(model.attrs[k]) for k in model.attrs}})
            
shutil.make_archive(shp_dir, 'zip', shp_dir)  
shutil.rmtree(shp_dir)

print('{} created\n'.format(shp_dir+'.zip'))


##########################################################
# NETCDF #################################################
##########################################################
shutil.copy(FILENAME, container)
print('Original netCDF file copied to {}\n'.format(container+'/'))


##########################################################
# ZIP IT ALL TOGETHER ####################################
##########################################################
shutil.make_archive(container, 'zip', container)
shutil.rmtree(container)
print('---\n')
print('{} created'.format(container+'.zip'))

In [ ]:
%matplotlib inline

import numpy as np
import simplekml
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LogNorm
import colorcet as cc
import xarray as xr
import xesmf as xe
from vis_tools import read_hysplit_netcdf, show_regridding_effects
import time
from pandas import Timestamp
import shutil

from contextlib import contextmanager
import sys
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

from pytz import timezone
NZT = timezone('Pacific/Auckland')


#################################################
# SPECIFY: regridding parameters
NEW_CELL_SIZE = 0.1  # side length of cell in degrees
ALG = 'bilinear'  # regridding algorithm

# SPECIFY: whether or not to include GIS products
INCLUDE_DATA = False
#################################################


data_zip = FILENAME.rstrip('.nc')+'.zip'
if INCLUDE_DATA == True & (not os.path.exists(data_zip)):    
    sys.exit('No data ZIP file found')

    
# define the colormap styles
CMAP = cc.m_linear_kry_5_95_c72_r
mapper = ScalarMappable(norm=LogNorm(vmin=ASH_MIN, vmax=ASH_MAX), cmap=CMAP)
cmap_rgba = CMAP(np.arange(CMAP.N), bytes=True)
cmap_kml_hex = [simplekml.Color.rgb(*rgba) for rgba in cmap_rgba] 

stylemaps = []
for kml_hex in cmap_kml_hex:
    
    alpha = 0.5  # specified as a float between 0 and 1
    color_with_alpha = simplekml.Color.changealphaint(int(alpha*255), kml_hex) 
    
    sm = simplekml.StyleMap()
    
    sm.normalstyle.polystyle.outline = 0
    sm.normalstyle.iconstyle.scale = 0
    sm.normalstyle.iconstyle.icon = None
    sm.normalstyle.iconstyle.heading = None
    sm.normalstyle.iconstyle.colormode = None
    sm.normalstyle.labelstyle.scale = 0
    sm.normalstyle.labelstyle.colormode = None
    sm.normalstyle.polystyle.color = color_with_alpha
    sm.normalstyle.polystyle.colormode = None
     
    sm.highlightstyle.polystyle.outline = None  # this ENABLES the outline
    sm.highlightstyle.iconstyle.scale = 0
    sm.highlightstyle.iconstyle.icon = None
    sm.highlightstyle.iconstyle.heading = None
    sm.highlightstyle.iconstyle.colormode = None
    sm.highlightstyle.labelstyle.scale = 1  # make label appear when cell is hovered over
    sm.highlightstyle.labelstyle.colormode = None
    sm.highlightstyle.polystyle.color = color_with_alpha
    sm.highlightstyle.polystyle.colormode = None

    sm.highlightstyle.linestyle.width = 3  # make an outline appear when cell is hovered over
    sm.highlightstyle.linestyle.colormode = None
        
    stylemaps.append(sm)

shared_styles = dict(zip(cmap_kml_hex, stylemaps))  


# read in and regrid the model
model = read_hysplit_netcdf(FILENAME)
new_grid = xr.Dataset({'lat': (['lat'], np.arange(np.min(model['lat']), np.max(model['lat']), NEW_CELL_SIZE)),
                       'lon': (['lon'], np.arange(np.min(model['lon']), np.max(model['lon']), NEW_CELL_SIZE))
                      })
with suppress_stdout():
    regridder = xe.Regridder(model, new_grid, ALG)
    td_regrid = regridder(model['total_deposition'].transpose('time', 'lat', 'lon'))
    regridder.clean_weight_file()

show_regridding_effects(fname=FILENAME, csz=NEW_CELL_SIZE, alg=ALG)


# define model, cell bounds
td_regrid_bounds = dict(left=td_regrid['lon'].values[0] - NEW_CELL_SIZE/2,
                        right=td_regrid['lon'].values[-1] + NEW_CELL_SIZE/2,
                        bottom=td_regrid['lat'].values[0] - NEW_CELL_SIZE/2,
                        top=td_regrid['lat'].values[-1] + NEW_CELL_SIZE/2
                        )

cell_bounds = dict(x=np.linspace(td_regrid_bounds['left'], td_regrid_bounds['right'], len(td_regrid['lon'])+1),
                   y=np.linspace(td_regrid_bounds['bottom'], td_regrid_bounds['top'], len(td_regrid['lat'])+1)
                  )


# create a new KML file
kml = simplekml.Kml()


# create the volcano placemark
volc_loc = kml.newpoint(name=model.attrs['volcano'])
volc_loc.coords = [tuple(model.attrs['volcano_location'][::-1])]
volc_loc.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/volcano.png'

volc_info = ''
for key in model.attrs:
    volc_info = volc_info + ('{}: {}\n'.format(key, model.attrs[key]))
volc_loc.style.balloonstyle.text = volc_info


# add a colorbar legend
fig = plt.figure(figsize=(1, 4))
ax = fig.add_axes([0, 0, 0.3, 0.6])

cb = mpl.colorbar.ColorbarBase(ax, cmap=mapper.cmap, norm=mapper.norm)

cb.set_label('Ash thickness (mm)', rotation=-90, labelpad=15)
cb.set_ticks(np.logspace(np.log10(ASH_MIN), np.log10(ASH_MAX), np.log10(ASH_MAX)-np.log10(ASH_MIN)+1))
cb.set_ticklabels(['{:g}'.format(tick) for tick in cb.get_ticks()])

legend_fname = 'legend.png'
fig.savefig(legend_fname, dpi=110, pad_inches=0.1, bbox_inches='tight')
plt.close()

legend = kml.newscreenoverlay(name='Legend')
legend.icon.href = legend_fname
legend.overlayxy = simplekml.OverlayXY(x=1, y=0,
                                       xunits=simplekml.Units.fraction,
                                       yunits=simplekml.Units.fraction)
legend.screenxy = simplekml.ScreenXY(x=1, y=0.12,
                                     xunits=simplekml.Units.fraction,
                                     yunits=simplekml.Units.fraction)
legend.size.x, legend.size.y = [0, 0]
legend.size.xunits = simplekml.Units.fraction
legend.size.yunits = simplekml.Units.fraction    


# create a folder of cell values for each time step
for time_step in td_regrid['time'].values:
        
    fol = kml.newfolder(name=str(np.datetime64(time_step, 'h')))
    
    fol.timespan.begin = NZT.localize(Timestamp(time_step).to_pydatetime()).isoformat()
    ts = np.mean(np.ediff1d(td_regrid['time']).astype('timedelta64[h]'))
    fol.timespan.end = NZT.localize(Timestamp(time_step+ts).to_pydatetime()).isoformat()
    
    fol.style.liststyle.listitemtype = 'checkHideChildren'
    
    t_i = time.time()
    
    for x_ind in range(len(td_regrid['lon'])):
        for y_ind in range(len(td_regrid['lat'])):
            cell = {}

            value = td_regrid.sel(time=time_step).values[y_ind, x_ind]

            if np.isnan(value):
                pass
            else:        
                cell['left'] = cell_bounds['x'][x_ind]
                cell['right'] = cell_bounds['x'][x_ind+1]        
                cell['bottom'] = cell_bounds['y'][y_ind]
                cell['top'] = cell_bounds['y'][y_ind+1]
                cell['x'] = td_regrid['lon'].values[x_ind]        
                cell['y'] = td_regrid['lat'].values[y_ind]        
                
                # make value formatting more sensible
                if value < ASH_MIN:
                    val = '< {} mm'.format(ASH_MIN)
                elif value < 99.5:
                    val = '{:.2g} mm'.format(value)
                else:
                    val = '{:.3g} mm'.format(value)
                
                geo = fol.newmultigeometry(name=val)

                pnt = geo.newpoint(altitudemode='absolute')
                poly = geo.newpolygon(altitudemode='clampToGround') 

                pnt.coords = [(cell['x'], cell['y'])]
                poly.outerboundaryis = [
                                        (cell['left'], cell['bottom']),
                                        (cell['right'], cell['bottom']),
                                        (cell['right'], cell['top']),
                                        (cell['left'], cell['top']),
                                        (cell['left'], cell['bottom'])
                                       ]

                rgba = mapper.to_rgba(value, bytes=True)
                kml_hex = simplekml.Color.rgb(*rgba)
                geo.stylemap = shared_styles[kml_hex]
                        
    print('{} folder written'.format(np.datetime64(time_step, 'h')))
    t_f = time.time()
    print('{:.2f} seconds elapsed\n'.format(t_f - t_i))

    
# specify startup params    
end_time = td_regrid.isel(time=-1)['time'].values
kml.document.lookat.gxtimestamp.when = NZT.localize(Timestamp(end_time+ts).to_pydatetime()).isoformat()
kml.document.lookat.altitude = 1100000  # m
kml.document.lookat.altitudemode = 'absolute'
kml.document.lookat.heading = 0
kml.document.lookat.tilt = 0
kml.document.lookat.latitude, kml.document.lookat.longitude = [-39, 176]


# add in the GIS data products if specified
if INCLUDE_DATA == True:
        
    shutil.copyfile(data_zip, 'data.zip')

    kml.addfile('data.zip')
    print('Data ZIP file added to KMZ file\n')

    kml.savekmz(FILENAME.rstrip('.nc')+'.kmz')
    os.remove('data.zip')
        
else:  # INCLUDE_DATA == False
    
    kml.savekmz(FILENAME.rstrip('.nc')+'.kmz')
    
os.remove(legend_fname)
print('{} created'.format(FILENAME.rstrip('.nc')+'.kmz'))